# Proyecto Integrador — Módulo 4: Preparación de Datos (NumPy + Pandas)
## Contexto
Este proyecto simula el flujo de trabajo del equipo de analítica de datos de una empresa e-commerce, preparando un dataset a partir de múltiples fuentes y resolviendo problemas comunes de calidad de datos (nulos, duplicados, formatos inconsistentes y outliers).

## Objetivo
Desarrollar un proceso reproducible para:

Generar datos ficticios con NumPy.
Explorar y transformar datos con Pandas.
Integrar fuentes (CSV, Excel y web).
Limpiar nulos y tratar outliers con técnicas estadísticas.
Aplicar data wrangling, agrupamientos y pivoteo.
Exportar el dataset final a CSV y Excel.

In [13]:
# Proyecto Integrador - Modulo 4: Preparación de Datos

import numpy as np
import pandas as pd




## Lección 1 — Generación de datos con NumPy
¿Por qué NumPy es eficiente para datos numéricos?
NumPy es eficiente porque trabaja con arrays homogéneos en memoria continua y aplica operaciones vectorizadas (sin bucles de Python), lo que permite:

Mayor velocidad en cálculos (media, suma, conteos, etc.).
Menor consumo de memoria comparado con listas de Python.
Facilidad para generar datos sintéticos con np.random.
En esta etapa se generan clientes ficticios y se introducen intencionalmente valores problemáticos (un nulo y un outlier) para aplicar limpieza en etapas posteriores.



In [14]:

# --- LECCIÓN 1: GENERACIÓN DE DATOS CON NUMPY ---
# Vamos a generar 10 clientes extra para simular que llegan datos nuevos

np.random.seed(42)
ids_extra = np.arange(101, 111) # IDs del 101 al 110
edades_extra = np.random.randint(18, 65, size=10)
compras_extra = np.random.randint(1, 15, size=10)
montos_extra = np.random.uniform(500, 5000, size=10)

# Introducimos un nulo y un outlier para cumplir con la limpieza posterior
montos_extra[0] = np.nan
montos_extra[1] = 50000 # Outlier exagerado

# Operaciones básicas
print(f"Monto promedio (sin contar nulos): {np.nanmean(montos_extra)}")

# Guardamos
datos_extra = np.column_stack((ids_extra, edades_extra, compras_extra, montos_extra))
np.save('datos_extra.npy', datos_extra)
print("✅ Lección 1: Datos generados.")



Monto promedio (sin contar nulos): 7588.444678741504
✅ Lección 1: Datos generados.


## Lección 2 — Exploración inicial con Pandas
En esta etapa se convierte el array de NumPy en un DataFrame, lo que permite:

Inspección rápida con head(), describe() y filtros.
Manipulación tabular (columnas, tipos, transformaciones).
Exportación a formatos comunes para análisis (.csv).
Se genera un CSV preliminar para usarlo como insumo en la integración de fuentes de la Lección 3.

In [15]:
# --- LECCIÓN 2: EXPLORACIÓN CON PANDAS ---

# Convertimos los datos de NumPy a DataFrame
df_extra = pd.DataFrame(datos_extra, columns=['ID', 'Edad', 'Total_Compras', 'Monto_Total'])

# Agregamos nombres ficticios para que coincida con tus otros archivos
df_extra['Nombre'] = ['Cliente_Extra_' + str(i) for i in range(1, 11)]
df_extra['Ciudad'] = 'Santiago' # Ciudad por defecto

# Exploración
print(df_extra.head())
df_extra.to_csv('clientes_extra.csv', index=False)
print("✅ Lección 2: DataFrame creado y exportado.")

      ID  Edad  Total_Compras   Monto_Total           Nombre    Ciudad
0  101.0  56.0           11.0           NaN  Cliente_Extra_1  Santiago
1  102.0  46.0            8.0  50000.000000  Cliente_Extra_2  Santiago
2  103.0  32.0            5.0    503.504446  Cliente_Extra_3  Santiago
3  104.0  60.0            4.0   4964.952017  Cliente_Extra_4  Santiago
4  105.0  25.0            8.0   3278.666793  Cliente_Extra_5  Santiago
✅ Lección 2: DataFrame creado y exportado.


## Lección 3 — Obtención de datos desde múltiples fuentes (CSV, Excel y web)
En esta etapa se integran tres fuentes:

clientes_ecommerce.csv
clientes_ecommerce.xlsx
clientes_extra.csv (proveniente de los datos generados con NumPy)
Desafío encontrado: al intentar extraer una tabla web con pd.read_html(), la web respondió con un error HTTP 403: Forbidden.
Para evitar que el pipeline falle, se implementó un manejo de errores (try/except) y se continuó con las fuentes locales. Esto es importante en escenarios reales, donde las fuentes web pueden bloquear bots o cambiar su estructura.

El resultado de esta etapa es un dataset consolidado con 30 registros (antes de limpieza), listo para tratar nulos, duplicados y outliers.

In [16]:
# --- LECCIÓN 3: OBTENCIÓN DE DATOS DESDE ARCHIVOS ---

# 1. Leer el CSV y el Excel que ya tienes en la carpeta
df_csv = pd.read_csv('clientes_ecommerce.csv')
df_excel = pd.read_excel('clientes_ecommerce.xlsx')

# 2. Leer datos desde una página web (Requerimiento técnico)
# Usaremos una tabla de Wikipedia con códigos de países o similar, solo para demostrar la técnica
try:
    url = "https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes"
    tablas = pd.read_html(url)
    df_web = tablas[0].iloc[:10, :2] # Tomamos solo un pedacito para el ejemplo
    print("✅ Datos web extraídos con éxito.")
except Exception as e:
    print(f"⚠️ No se pudo leer la web, pero continuaremos con los archivos locales. Error: {e}")

# 3. Unificar las fuentes locales (CSV, Excel y los datos de NumPy que creamos)
# Primero cargamos el que creamos en la Lección 2
df_extra = pd.read_csv('clientes_extra.csv')

# Concatenamos todos los DataFrames de clientes
df_consolidado = pd.concat([df_csv, df_excel, df_extra], ignore_index=True)

# 4. Guardar el DataFrame consolidado
df_consolidado.to_csv('clientes_consolidado.csv', index=False)

print(f"\n📊 Total de registros unificados: {len(df_consolidado)}")
print("\n--- Primeras filas del dataset consolidado ---")
print(df_consolidado.tail()) # Usamos tail para ver los que agregamos al final




⚠️ No se pudo leer la web, pero continuaremos con los archivos locales. Error: HTTP Error 403: Forbidden

📊 Total de registros unificados: 30

--- Primeras filas del dataset consolidado ---
       ID            Nombre  Edad    Ciudad  Total_Compras  Monto_Total
25  106.0   Cliente_Extra_6  38.0  Santiago            8.0  3252.439222
26  107.0   Cliente_Extra_7  56.0  Santiago            3.0   531.798373
27  108.0   Cliente_Extra_8  36.0  Santiago            6.0   603.780913
28  109.0   Cliente_Extra_9  40.0  Santiago            5.0  2861.485971
29  110.0  Cliente_Extra_10  28.0  Santiago            2.0  2299.374373


## Lección 4 — Valores perdidos (nulos) y outliers
Valores nulos
Se identificaron nulos en:

Edad (2 registros)
Monto_Total (1 registro)
Decisión de imputación:

Para Monto_Total se utilizó la mediana, porque es más robusta ante valores extremos (outliers).
Para Edad se utilizó la media, ya que es una variable numérica donde la imputación es razonable para no perder registros.
Outliers
Se detectaron outliers en Monto_Total usando el método IQR (rango intercuartílico).
Se encontró 1 outlier principal (un monto extremadamente alto generado intencionalmente).

Tratamiento aplicado: capping (recorte) al límite superior del IQR, en lugar de eliminar el registro, para conservar información y evitar distorsión en estadísticas.


In [17]:
# --- LECCIÓN 4: MANEJO DE VALORES PERDIDOS Y OUTLIERS ---

# 1. Identificar valores nulos
print("🔍 Valores nulos por columna:")
print(df_consolidado.isnull().sum())

# 2. Gestión de nulos: Imputación
# Vamos a llenar los montos nulos con la mediana (es más segura que el promedio)
mediana_monto = df_consolidado['Monto_Total'].median()
df_consolidado['Monto_Total'] = df_consolidado['Monto_Total'].fillna(mediana_monto)

# También hay nulos en Edad (del archivo original), los llenamos con la media
df_consolidado['Edad'] = df_consolidado['Edad'].fillna(df_consolidado['Edad'].mean())

# 3. Detección de Outliers usando IQR (Rango Intercuartílico)
Q1 = df_consolidado['Monto_Total'].quantile(0.25)
Q3 = df_consolidado['Monto_Total'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Identificamos quiénes son los outliers
outliers = df_consolidado[(df_consolidado['Monto_Total'] < limite_inferior) | (df_consolidado['Monto_Total'] > limite_superior)]
print(f"\n🚨 Se detectaron {len(outliers)} outliers en Monto_Total.")
print(outliers[['Nombre', 'Monto_Total']])

# 4. Tratamiento de Outliers: Los limitamos al valor del límite superior (Capping)
df_consolidado.loc[df_consolidado['Monto_Total'] > limite_superior, 'Monto_Total'] = limite_superior

# 5. Guardar DataFrame limpio
df_consolidado.to_csv('clientes_limpio.csv', index=False)
print("\n✅ Lección 4 completada: Datos limpios y sin outliers extremos.")



🔍 Valores nulos por columna:
ID               0
Nombre           0
Edad             2
Ciudad           0
Total_Compras    0
Monto_Total      1
dtype: int64

🚨 Se detectaron 1 outliers en Monto_Total.
             Nombre  Monto_Total
21  Cliente_Extra_2      50000.0

✅ Lección 4 completada: Datos limpios y sin outliers extremos.


## Lección 5 — Data Wrangling (transformación y enriquecimiento)
En esta etapa se aplicaron técnicas para dejar el dataset más útil para análisis:

Eliminación de duplicados: al combinar CSV y Excel se detectaron 10 registros duplicados (mismo contenido), que fueron eliminados para evitar doble conteo.
Conversión de tipos: se aseguraron tipos correctos en columnas clave (por ejemplo, ID y Edad como enteros).
Creación de nuevas variables:
Ticket_Promedio = Monto_Total / Total_Compras
Categoria_Cliente (Regular / Premium) según gasto total.
Rango_Etario (Joven / Adulto / Senior) usando discretización con pd.cut.
Nota de calidad de datos: el contacto “Carlos” tiene Total_Compras = 0, por lo que Ticket_Promedio resulta NaN (división por cero). Esta situación refleja un caso real: cuando no hay compras, el ticket promedio no aplica y debe tratarse en análisis posterior (por ejemplo, con reglas de negocio o reemplazo condicionado).


In [18]:
# --- LECCIÓN 5: DATA WRANGLING ---

# 1. Eliminar registros duplicados (si existieran por la unión de archivos)
antes = len(df_consolidado)
df_consolidado = df_consolidado.drop_duplicates()
print(f"Registros eliminados por duplicidad: {antes - len(df_consolidado)}")

# 2. Transformar tipos de datos
# Aseguramos que ID y Edad sean enteros (a veces quedan como float por los nulos)
df_consolidado['ID'] = df_consolidado['ID'].astype(int)
df_consolidado['Edad'] = df_consolidado['Edad'].astype(int)

# 3. Crear una nueva columna calculada: 'Ticket_Promedio'
# Es el monto total dividido por el total de compras
df_consolidado['Ticket_Promedio'] = df_consolidado['Monto_Total'] / df_consolidado['Total_Compras']

# 4. Aplicar funciones personalizadas (lambda)
# Vamos a categorizar a los clientes según su gasto
# Si gasta más de 3000 es "Premium", si no "Regular"
df_consolidado['Categoria_Cliente'] = df_consolidado['Monto_Total'].apply(lambda x: 'Premium' if x > 3000 else 'Regular')

# 5. Normalización / Discretización (Requerimiento técnico)
# Vamos a crear rangos de edad (Discretización)
bins = [0, 30, 50, 100]
labels = ['Joven', 'Adulto', 'Senior']
df_consolidado['Rango_Etario'] = pd.cut(df_consolidado['Edad'], bins=bins, labels=labels)

# Guardar versión optimizada
df_consolidado.to_csv('clientes_optimizado.csv', index=False)

print("\n✅ Lección 5 completada: Datos transformados y enriquecidos.")
print(df_consolidado[['Nombre', 'Ticket_Promedio', 'Categoria_Cliente', 'Rango_Etario']].head())



Registros eliminados por duplicidad: 10

✅ Lección 5 completada: Datos transformados y enriquecidos.
   Nombre  Ticket_Promedio Categoria_Cliente Rango_Etario
0     Ana            500.0           Regular        Joven
1    Luis            600.0           Regular       Adulto
2  Carlos              NaN           Regular       Adulto
3   Marta            600.0           Regular        Joven
4   Jorge            400.0           Premium       Adulto


## Lección 6 — Agrupamiento, pivoteo y exportación final
Para preparar el dataset para análisis y reportes:

Se generó un resumen por ciudad con groupby() (métricas como gasto promedio, total de compras y cantidad de clientes).
Se construyó una tabla pivote (pivot_table) para observar el gasto promedio cruzando Rango_Etario y Categoria_Cliente.
Entregables generados
Dataset final exportado en:
dataset_final_ecommerce.csv
dataset_final_ecommerce.xlsx
Con esto se completa el flujo integral de preparación de datos: generación, integración, limpieza, transformación y estructuración para análisis.


In [19]:
# --- LECCIÓN 6: AGRUPAMIENTO Y PIVOTEO DE DATOS ---

# 1. Agrupamiento (groupby) para obtener métricas por Ciudad
# Calculamos el promedio de gasto y el total de compras por ciudad
reporte_ciudad = df_consolidado.groupby('Ciudad').agg({
    'Monto_Total': 'mean',
    'Total_Compras': 'sum',
    'ID': 'count'
}).rename(columns={'ID': 'Cantidad_Clientes'})

print("📊 Reporte por Ciudad:")
print(reporte_ciudad)

# 2. Reestructuración de datos: Pivot Table
# Vamos a ver el Monto_Total promedio cruzando Rango_Etario y Categoria_Cliente
pivot_reporte = df_consolidado.pivot_table(
    values='Monto_Total', 
    index='Rango_Etario', 
    columns='Categoria_Cliente', 
    aggfunc='mean',
    observed=True # Para evitar advertencias con categorías vacías
)

print("\nTabla Pivote (Gasto promedio por Rango Etario y Categoría):")
print(pivot_reporte)

# 3. Exportación Final (Requerimiento obligatorio)
df_consolidado.to_csv('dataset_final_ecommerce.csv', index=False)
df_consolidado.to_excel('dataset_final_ecommerce.xlsx', index=True)

print("\n✅ Lección 6 completada: Reportes generados y archivos finales exportados.")
print("📁 Archivos listos: 'dataset_final_ecommerce.csv' y 'dataset_final_ecommerce.xlsx'")




📊 Reporte por Ciudad:
              Monto_Total  Total_Compras  Cantidad_Clientes
Ciudad                                                     
Bahía Blanca  2950.000000            6.0                  1
Buenos Aires  2500.000000            5.0                  1
Córdoba       1200.000000            2.0                  1
La Plata      3200.000000            8.0                  1
Mendoza       1800.000000            3.0                  1
Neuquén       3300.000000            7.0                  1
Rosario          0.000000            0.0                  1
Salta         4100.000000           10.0                  1
Santa Fe      2100.000000            4.0                  1
Santiago      2709.432565           60.0                 10
Tucumán        400.000000            1.0                  1

Tabla Pivote (Gasto promedio por Rango Etario y Categoría):
Categoria_Cliente      Premium      Regular
Rango_Etario                               
Joven              3689.333397  2387.343593
Adult

## Posibles mejoras (a futuro)
Implementar validaciones adicionales (por ejemplo, evitar Total_Compras = 0 si el negocio lo exige, o manejarlo con una regla explícita).
Registrar el pipeline en funciones reutilizables (modularización) y/o logging para trazabilidad.
Usar fuentes web alternativas o APIs con autenticación para evitar bloqueos 403.
Agregar tests simples (por ejemplo, validar que no queden nulos en columnas críticas luego de la limpieza).
